In [1]:
!rm -r /kaggle/working/*
%cd /kaggle/working

/kaggle/working


In [2]:
import os
import sys

PACKAGE_DIR = "/kaggle/src"
sys.path.append(PACKAGE_DIR)
sys.path.append(os.path.join(PACKAGE_DIR, "Penguin-ML-Library"))

In [3]:
import yaml
from penguinml.utils.logger import get_logger, init_logger
from penguinml.utils.set_seed import seed_base

MODEL_NAME = "lightgbm"
CFG = yaml.safe_load(open(os.path.join(PACKAGE_DIR, "config.yaml"), "r"))
print(CFG[MODEL_NAME]["execution"]["exp_id"])
CFG["output_dir"] = f"/kaggle/output/{CFG[MODEL_NAME]['execution']['exp_id']}"
!rm -r {CFG["output_dir"]}
os.makedirs(CFG["output_dir"], exist_ok=True)

init_logger(f"{ CFG[MODEL_NAME]['execution']['exp_id']}.log")
logger = get_logger("main")
seed_base(CFG[MODEL_NAME]["execution"]["seed"])

2024-11-16 13:50:59.774433: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-16 13:50:59.800056: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


exp_006


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
set seed: 46


In [4]:
import warnings

import numpy as np
import polars as pl
from IPython.display import clear_output, display
from penguinml.gbdt.lightgbm import fit_lgb, inference_lgb
from tqdm import tqdm

from const import TARGET_COLS
from feature_engineering import feature_engineer

warnings.filterwarnings("ignore")
pl.Config.set_tbl_rows(20)

train = pl.read_csv(os.path.join(CFG["dataset"]["competition_dir"], "train_features.csv"))
train = (
    train.with_columns(
        pl.col("ID").str.split_exact("_", n=1).struct.rename_fields(["sceneID", "offset"]).alias("fields")
    )
    .unnest("fields")
    .with_columns(pl.col("offset").cast(pl.Float32))
)
print(train.shape)
train.head(1)

(43371, 32)


ID,vEgo,aEgo,steeringAngleDeg,steeringTorque,brake,brakePressed,gas,gasPressed,gearShifter,leftBlinker,rightBlinker,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5,sceneID,offset
str,f64,f64,f64,f64,f64,bool,f64,bool,str,bool,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f32
"""00066be8e20318869c38c66be46663…",5.701526,1.538456,-2.165777,-139.0,0.0,false,0.25,true,"""drive""",false,false,2.82959,0.032226,0.045187,6.231999,0.065895,0.107974,9.785009,0.124972,0.203649,13.485472,0.163448,0.302818,17.574227,0.174289,0.406331,21.951269,0.199503,0.485079,"""00066be8e20318869c38c66be46663…",320.0


## 特徴量生成


In [5]:
train, features = feature_engineer(train)

## CV Split


In [6]:
train_folds = pl.read_csv(CFG["dataset"]["train_fold_path"])
train = train.join(train_folds, on="sceneID", how="left")
assert train["fold"].null_count() == 0

## Training


In [ ]:
dfs = []
scores = []
for c in tqdm(sorted(TARGET_COLS, key=lambda x: x.split("_")[1], reverse=True)):
    this_df = train.filter(pl.col("target_name_original") == c)
    oof, models = fit_lgb(
        data=this_df,
        features=features,
        params=CFG[MODEL_NAME]["params"],
        target_col="target",
        fold_col="fold",
        target_type="regression",
        verbose=50000,
    )
    this_df = this_df.with_columns(pl.Series("oof", oof))
    dfs.append(this_df)

    i = int(c.split("_")[1])
    xyz = c.split("_")[0]
    mae = np.abs(this_df["oof"] - this_df["target"]).mean()
    score = {"target": c, "cv": mae} | {
        f"fold{fold}": np.abs(
            this_df.filter(pl.col("fold") == fold)["oof"] - this_df.filter(pl.col("fold") == fold)["target"]
        ).mean()
        for fold in range(5)
    }
    scores.append(score)

    clear_output()
    display(pl.DataFrame(scores).sort("target"))

train = pl.concat(dfs)
clear_output()
display(pl.DataFrame(scores).sort("target"))

target,cv,fold0,fold1,fold2,fold3,fold4
str,f64,f64,f64,f64,f64,f64
"""x_0""",0.063883,0.063559,0.062202,0.065506,0.063717,0.064429
"""x_1""",0.137001,0.135529,0.135043,0.138745,0.136518,0.139172
"""x_2""",0.232062,0.229925,0.230675,0.232332,0.229519,0.237857
"""x_3""",0.360112,0.356194,0.364038,0.360692,0.353767,0.365868
"""x_4""",0.513969,0.505727,0.519416,0.514782,0.51125,0.518673
"""x_5""",0.690449,0.682297,0.699715,0.688853,0.682781,0.6986
"""y_0""",0.032201,0.031942,0.031407,0.032722,0.031838,0.033096
"""y_1""",0.073227,0.072919,0.07085,0.074316,0.072983,0.07507
"""y_2""",0.131989,0.129664,0.129739,0.133781,0.131383,0.13538


In [8]:
mae = np.abs(train["oof"] - train["target"]).mean()
print(f"MAE: {mae}")

MAE: 0.2149617660813801


In [9]:
oof_df = (
    train.select(["ID", "target_name_original", "oof"])
    .with_columns(pl.col("target_name_original"))
    .pivot(index="ID", columns="target_name_original", values="oof")
)
oof_df.head()

ID,x_5,y_5,z_5,x_4,y_4,z_4,x_3,y_3,z_3,x_2,y_2,z_2,x_1,y_1,z_1,x_0,y_0,z_0
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""00066be8e20318869c38c66be46663…",22.333394,-0.495721,0.170445,17.974801,-0.307298,0.122692,13.554547,-0.13487,0.086634,9.204076,-0.022538,0.069425,5.938599,-0.033567,0.023516,2.689897,-0.00109,0.017039
"""00066be8e20318869c38c66be46663…",32.103596,-0.770213,0.151166,26.710388,-0.718927,0.117289,21.452011,-0.384485,0.106703,16.046806,-0.403307,0.080189,10.593637,-0.241345,0.032359,5.046198,-0.074474,0.023015
"""00066be8e20318869c38c66be46663…",32.494253,-0.262193,0.161021,26.561009,-0.260789,0.117969,20.986193,-0.154247,0.114744,15.655412,-0.075275,0.080898,10.146153,-0.056597,0.033953,4.774744,-0.025693,0.022368
"""000fb056f97572d384bae4f5fc1e0f…",14.445091,-0.763033,0.080021,11.329737,-0.521458,0.046855,8.814815,-0.357669,0.035565,5.838932,-0.246986,0.020457,3.700123,-0.116017,0.009566,1.616424,-0.054994,0.002238
"""000fb056f97572d384bae4f5fc1e0f…",17.821408,1.2289,-0.034571,15.286568,0.829962,-0.024545,11.962893,0.569597,-0.015602,8.838489,0.281422,-0.007811,5.789547,0.109082,-0.000321,2.798834,0.019786,-0.001079


In [10]:
oof_df.write_csv(os.path.join(CFG["output_dir"], "oof.csv"))